# Setup

In [1]:
import fastai
from fastai.text import *
from sentencepiece import SentencePieceProcessor

In [2]:
wgts_path  = "./work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
model_path = "./work/up_low50k/tmp/sp-50k.model"

processor = SentencePieceProcessor()
processor.Load(model_path)
processor.SetEncodeExtraOptions("bos:eos")
processor.SetDecodeExtraOptions("bos:eos");

In [3]:
bptt = 5
max_seq = 1000000
n_tok = len(processor)
emb_sz = 400
n_hid = 1150
n_layers = 4
pad_token = 1
bidir = False
qrnn = False

rnn = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token,
                    bidir, qrnn)

model = SequentialRNN(rnn, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=rnn.encoder))

load_model(model[0], wgts_path)
model.reset()
model.eval();

In [8]:
class TextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]


def next_word(sentence, model): 
    ids = [np.array(processor.encode_as_ids(sentence))]

    dataset = TextDataset(ids)
    sampler = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(dataset,
                    batch_size=100,
                    transpose=True,
                    pad_idx=1,
                    sampler=sampler,
                    pre_pad=False)

    tensors = None
    with no_grad_context():
        for (x, y) in dl:
            tensors, _, _ = model(x)
            
    last_tensor = tensors[-1]

    best = int(torch.argmax(last_tensor))
    word = processor.decode_ids([best])
    
    while best in ids[0] or not word.isalpha():
        last_tensor[best] = -1
        best = int(torch.argmax(last_tensor))
        word = processor.decode_ids([best])

    return word

In [9]:
def continuation(sentence, model, word_count):
    result = ""
    for _ in range(word_count):
        word = next_word(sentence + " " + result, model)

        if len(word) > 0 and word[0].isalpha() and (
                len(word) > 1 or word.lower() in ['a', 'o', 'i', 'w', 'z']):
            result += (" " + word)
        else:
            result += word
    return result

# 4

In [10]:
sentences = [
  "Warszawa to największe",
  "Te zabawki należą do",
  "Policjant przygląda się",
  "Na środku skrzyżowania widać",
  "Właściciel samochodu widział złodzieja z",
  "Prezydent z premierem rozmawiali wczoraj o",
  "Witaj drogi",
]

In [12]:
for sentence in sentences:
    print("||" + sentence + "||" + continuation(sentence, model, 45))
    print("\n\n")

||Warszawa to największe|| miasto w Polsce i na świecie jest największym miastem Polski oraz jednym z największ ych miast regionu kujawsko o środek ciężkości ruchu turystycznego do centrum miasta Katowice przy ule Grunwaldzk i i i i i i i i i i i i i i



||Te zabawki należą do|| najbardziej popularnych i cenion ych w Polsce oraz na świecie z powodzeniem są dostępne dla najmłodszych dzieci podczas wakacj ów ch łód ek wi aer tyr omer type de la vi am elr ijk en



||Policjant przygląda się|| temu i próbuje go zatrzymać w spokoju na miejscu oraz po rwać z łóżka jego ciało lub głowę jak to możliwe że nie ma nic do straceni a nawet za sypia nie chce mu przeszkadzać ani też pić alkoholu czy narkotyków albo papieros a także



||Na środku skrzyżowania widać|| ślady po sobie i w okolicy oraz na ścianach bocznych kościoła św i Jana Chrzciciel a takżeu ch rz z lat pięćdziesiątych XX wieku o tematyce religijn ej am ner o osób świecki mi ał się



||Właściciel samochodu widział złodzieja z|| zami

# 5

In [57]:
questions = [
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie"
]

In [58]:
for sentence in questions:
    print("||" + sentence + "||" + continuation(sentence, model, 70))
    print("\n\n")

||Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie|| spodziewał że w końcu będzie sz lić i tak jak ja mam do czynienia z tym problemem a może nawet za jakiś czas będę miał problemy ze zdrowiem czy też po prostu mi sie uda bo jestem na urlopie macierzyński ym a więc jest bardzo dobrze ale chyba już dawno nic by mnie o bolał o niebo bardziej niż mój tata który był chory psychicznie od rana aż



||Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie|| spodziewał a ja mam w ielką ochotę na coś takiego jak np z ajebist ym i tak dalej ale chyba jest jeszcze gorzej niż przed wojną bo już dawno mi sie znudził o ile pamiętam ten ostatni był bardzo słaby do tego stopnia że po kilku latach gry zmienił em piku za darmo więc może być lepiej od niego tylko przez internet czy też grę online lub





# 6

In [13]:
text = "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"

In [16]:
print("||" + text + "||" + continuation(text, model, 70))

||Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane|| przez niego jako podgatun ce zarż nięty ch ibr ów osób o imieniu racował lub jego przodku oraz ich potomk om ni os wiadczeń do nich nie jest ani słowem nic zł ego albo co innego niż te które są używane przy ustalani ome ac tio di stri a cc ac ad di dic tum o a


# Other

In [24]:
jokes = [
  "Chuck Norris",
  "Co Chuck Norris",
  "Co Chuck Norris zrobi",
  "Chuck Norris to",
  "Chuck Norris jest",
  "Tylko Chuck Norris potrafi",
  "Przychodzi baba",
  "Przychodzi baba do",
  "Przychodzi baba do lekarza, a",
  "Przychodzi zezowata baba do",
  "Przychodzi baba do lekarza i mówi"
]

In [31]:
for sentence in jokes:
    print("||" + sentence + "||" + continuation(sentence, model, 45))
    print("\n\n")

||Chuck Norris|| jest żonaty i ma dwoje dzieci z pierwszego małżeństwa oraz syna bliźniak a także córkę Ann ettes Jr i siostrę Elizabeth Margaret Thatcher w wieku dwudziestu pięciu lat po ślubie ze swoją siostrą Karoliną We sley woody and the world



||Co Chuck Norris|| ma w głowie i jak a kompanie są na prawde to nie ch cac ej a ty zyj ał się osód ło tu po ci em o tym co jest dla ciebie najważniejsze ze wszystkich ludzi do



||Co Chuck Norris zrobi|| w tym roku to będzie już na pewno lepiej niż teraz i nie ch cac ej a ja mam nadzieję że się uda do tego czasu z nów po prostu za bije mnie jak o brzyd otyń bo jest tak samo dobrze ale



||Chuck Norris to|| nie tylko muzyka i taniec ale także rozrywk a przede wszystkim zabawa w rytm muzyki klasycznej oraz tańce ludowe z elementami jazzu czy folku na żywo lub koncert ów zespół muzyczny ze Szczecina do Krakowa po polsku bądź też zagranicą za granicą przez



||Chuck Norris jest|| żonaty i ma dwoje dzieci z pierwszego małżeństwa oraz syna 